<div style="text-align: right"> Brian Wiley <br/>
EN.705.601.3VL.SP20 Applied Machine Learning </div>

## Assignment 9
### Applied Machine Learning

__1. [10 pts] Describe the environment in the Nim learning model.__

The environment is the set of states from which the Nim learner interacts with by taking objects from pile in those states.  Simply the environment is the game of Nim where players choose to select objects from a pile in a series of choices to win the game.  Included in this q-learning environment is also a q-table from which the q-learner uses as a quote unquote cheat sheet to determine from memory what choices they have been made in the past that were good or bad.

__2. [10 pts] Describe the agent in the Nim learning model.__

As precluded above, the agent of learning model is a player, in our case the Nim learner player.  The agent is instructed what the goal of the game is which is to win the game.  In our model we don't actually tell the Nim learner directly that clearing the last pile is the goal we do this by adding rewards and penalties if it won or lost.  Which I will go into below.

__3. [10 pts] Describe the reward and penalty in the Nim learning model.__

The reward and penalties can only be distributed once the goal of the game is reached, i.e. we have a winner.  As indicated in our textbook, because during the game of Nim we cannot know when a particular move is bad or good.  Therefore the reward/penalty is distributed after all piles are empty.  The learner from lecture only distributed a reward in which that state and specific action at the state to win the game is rewarded.  I adapted to include a penalty which will be described in section 6.

__4. [10 pts] How many possible states there could be in the Nim game with a maximum of 10
items per pile and 3 piles total?__

Each pile can have between 0 and 10 objects so the total states is $11^{3}$ or 1,331.  This from the permuation formula where we have and order but can have repetition, i.e. 1,8,1 and 8,1,1 is the same permutation with different order.  We use the power rule to come up with the total.

__5. [10 pts] How many possible actions there could be in the Nim game with 10 items per pile
and 3 piles total?__

For each of the three piles as each pile could contain 1 or 2 or 3 or 4 or 5 or 6 or 7 or 8 or 9 or 10 objects and the player can take as many objects from only of the piles there are 10 possible actions per pile or 30 total action for the Nim game.

__6. [50 pts] Find a way to improve the provided Nim game learning model. Do you think one
can beat the Guru player? (Hint: How about penalizing the losses? Hint: It is indeed
possible to find a better solution which improves the way Q-learning updates its Q-table).__

I added a new function for nim_qlearn after the final qtable_log function at the bottom.  To sum up I found [resource](https://cdn.cs50.net/ai/2020/spring/projects/4/nim.zip) where in the train model for the q-learner, the learner is trained by playing games against itself.  We can keep track of player 0 and player 1 with a dictionary which keeps track of their last moves only.  When the game is one, we reward the state and action completed by the winner, i.e. the state before [0, 0, 0] and what pile and how many object chosen to win, and penalize the state and action that was completed last by the loser.  In actuality we would want to use graph theory here so that we could complete a DFS traverse in the reverse order possibly by using a stack or topological sorting as well to reward and penalize more than one state and action per winner and loser respectively.  

In [1]:
import numpy as np
from random import randint, choice

# The number of piles is 3


# max number of items per pile
ITEMS_MX = 10

# Initialize starting position
def init_game():
    return [randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

# Based on X-oring the item counts in piles - mathematical solution
def nim_guru(st):
    xored = st[0] ^ st[1] ^ st[2]
    if xored == 0:
        return nim_random(st)
    #
    for pile in range(3):
        s = st[pile] ^ xored
        if s <= st[pile]:
            return st[pile]-s, pile

# Random Nim player
def nim_random(_st):
    pile = choice([i for i in range(3) if _st[i]>0])  # find the non-empty piles
    return randint(1, _st[pile]), pile  # random move

In [2]:
[randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

[2, 10, 7]

In [3]:
def nim_qlearner(_st):
    # pick the best rewarding move
    a = np.argmax(qtable[_st[0], _st[1], _st[2]])
    # index is based on move, pile
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    # check if qtable has generated a random but game illegal move - we have not explored there yet
    if move <= 0 or _st[pile] < move :
        move, pile = nim_random(_st)
    #
    return move, pile

In [4]:
Engines = {'Random':nim_random, 'Guru':nim_guru, 'Qlearner':nim_qlearner}

def game(a, b):
    state, side = init_game(), 'A'
    while True:
        engine = Engines[a] if side == 'A' else Engines[b]
        move, pile = engine(state)
        # print(state, move, pile)  # debug purposes
        state[pile] -= move
        if state == [0, 0, 0]:  # game ends
            return side  # winning side
        #
        side = 'B' if side == 'A' else 'A'  # switch sides

def play_games(_n, a, b):
    from collections import defaultdict
    wins = defaultdict(int)
    for i in range(_n):
        wins[game(a, b)] += 1
    # info
    print(f"{_n} games, {a:>8s} {wins['A']}   {b:>8s} {wins['B']}")
    #
    return wins['A'], wins['B']

In [5]:
# Play games
play_games(1000, 'Random', 'Random')
play_games(1000, 'Guru', 'Random')
play_games(1000, 'Random', 'Guru')
play_games(1000, 'Guru', 'Guru') ;

1000 games,   Random 471     Random 529
1000 games,     Guru 996     Random 4
1000 games,   Random 9       Guru 991
1000 games,     Guru 945       Guru 55


In [6]:
qtable, Alpha, Gamma, Reward = None, 1.0, 0.8, 100.0

# learn from _n games, randomly played to explore the possible states
def nim_qlearn(_n):
    global qtable
    # based on max items per pile
    qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=float)
    # play _n games
    for i in range(_n):
        # first state is starting position
        st1 = init_game()
        while True:
            # make a random move - exploration
            move, pile = nim_random(st1)
            st2 = list(st1)
            # make the move
            st2[pile] -= move
            if st2 == [0, 0, 0]:  # game ends
                qtable_update(Reward, st1, move, pile, 0)
                break  # new game
            #
            qtable_update(0, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
            st1 = st2

# Equation 3 - update the qtable
def qtable_update(r, _st1, move, pile, q_future_best):
    a = pile*ITEMS_MX+move-1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)

In [7]:
%%time
nim_qlearn(100)

Wall time: 18 ms


In [8]:
# Play games
play_games(1000, 'Qlearner', 'Random')
play_games(1000, 'Random', 'Qlearner') ;

1000 games, Qlearner 703     Random 297
1000 games,   Random 313   Qlearner 687


In [9]:
%%time
# See the training size effect
n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
wins = []
for n in n_train:
    nim_qlearn(n)
    a, b = play_games(1000, 'Qlearner', 'Random')
    wins += [a/(a+b)]

1000 games, Qlearner 577     Random 423
1000 games, Qlearner 440     Random 560
1000 games, Qlearner 656     Random 344
1000 games, Qlearner 730     Random 270
1000 games, Qlearner 705     Random 295
1000 games, Qlearner 726     Random 274
1000 games, Qlearner 708     Random 292
Wall time: 15 s


In [10]:
print(wins)

[0.577, 0.44, 0.656, 0.73, 0.705, 0.726, 0.708]


In [11]:
def qtable_log(_fn):
    with open(_fn, 'w') as fout:
        s = 'state'
        for a in range(ITEMS_MX*3):
            move, pile = a%ITEMS_MX+1, a//ITEMS_MX
            s += ',%02d_%01d' % (move,pile)
        #
        print(s, file=fout)
        for i, j, k in [(i,j,k) for i in range(ITEMS_MX+1) for j in range(ITEMS_MX+1) for k in range(ITEMS_MX+1)]:
            s = '%02d_%02d_%02d' % (i,j,k)
            for a in range(ITEMS_MX*3):
                r = qtable[i, j, k, a]
                s += ',%.1f' % r
            #
            print(s, file=fout)
#
qtable_log('qtable_debug.txt')

### Here is the updated function.

In [12]:
qtable, Alpha, Gamma, Reward, Penalty = None, 1.0, 0.8, 100.0, -1000.0

# learn from _n games, randomly played to explore the possible states
def nim_qlearn_updated(_n):
    '''
    Nim q-learner is trained while playing games against inself
    Updates are adding last moves made by winner and loser and penalyzing last move by loser
    '''
    global qtable
    # based on max items per pile
    qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=float)
    # play _n games
    for i in range(_n):
        # first state is starting position
        st1 = init_game()
        
        ## this keeps track of the last move made by winner and loser
        last = {
            0: {"st": None, "pile": None, "move": None},
            1: {"st": None, "pile": None, "move": None}
        }
        ## used to switch q-learner players
        q_learner_player_num = 0
        
        while True:
            # make a random move - exploration
            move, pile = nim_random(st1)
            st2 = list(st1)
            # make the move
            st2[pile] -= move
            '''
            Switch player, we do this before checking if there is a winner
            so we get can the state, pile, and move from that was made last
            by the loser by penalizing last[q_learner_player_num] states
            and action.
            
            Example if player #0 removed the last object from the final pile
            then that state st1, move and pile from above gets rewarded. Then
            we switch the player number to #1 so we can get that for the penalty
            
            See Harvard's extension here https://cdn.cs50.net/ai/2020/spring/projects/4/nim.zip
            Note they have the goal switched and clearing last pile loses the game
            '''
            last[q_learner_player_num]["st"] = st1     ## current player's last state
            last[q_learner_player_num]["pile"] = pile
            last[q_learner_player_num]["move"] = move
            q_learner_player_num = 1 if q_learner_player_num == 0 else 0  ## switch player
    
            if st2 == [0, 0, 0]:  # game ends
                ## the current, i.e. st1, pile, move that was presnt and made by the winner
                qtable_update(Reward, st1, move, pile, 0)
                ## penalize the last move and action of player that lost
                ## i.e. the state and move before the state and move that won
                qtable_update(Penalty, last[q_learner_player_num]["st"],
                              last[q_learner_player_num]["move"],
                              last[q_learner_player_num]["pile"], 0)
                break  # new game
            # this should be an `else` to see visually it updates only if no winner
            else:
                qtable_update(0, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
            # update state
            st1 = st2
            

# Equation 3 - update the qtable
def qtable_update(r, _st1, move, pile, q_future_best):
    a = pile*ITEMS_MX+move-1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)

#### Now let's test to see if we get better at least better results.

In [15]:
%%time

no_penalty_wins = []
with_penalty_wins = []

## original no penalty train
nim_qlearn(10000000)
## test is 5 times for 1000 games each
for _ in range(5):
    no_penalty_wins.append(play_games(1000, 'Qlearner', 'Guru')[0])

## updated with penalty training
nim_qlearn_updated(10000000)
## test is 5 times for 1000 games each
for _ in range(5):
    with_penalty_wins.append(play_games(1000, 'Qlearner', 'Guru')[0])  

1000 games, Qlearner 22       Guru 978
1000 games, Qlearner 25       Guru 975
1000 games, Qlearner 33       Guru 967
1000 games, Qlearner 36       Guru 964
1000 games, Qlearner 26       Guru 974
1000 games, Qlearner 22       Guru 978
1000 games, Qlearner 20       Guru 980
1000 games, Qlearner 18       Guru 982
1000 games, Qlearner 22       Guru 978
1000 games, Qlearner 18       Guru 982
Wall time: 32min 9s


In [16]:
## print results
print(f'{"Without Penality":<25} {"With Penality":<15}')
for i in range(len(with_penalty_wins)):
    print(f'{no_penalty_wins[i]:^15} {with_penalty_wins[i]:^32}')

Without Penality          With Penality  
      22                       22               
      25                       20               
      33                       18               
      36                       22               
      26                       18               


I am not sure why but penalize the second to last move and state before the win actually makes the results worse.  I guess there is a better way to penalize but I can't think of one.